In [5]:
''' 
Klasa za zapis formule. 
Formula moze imati argumente i to bi bili operandi koji ucestvuju u formuli. 
Npr. za operaciju /\ imamo dva operanda.

Za iskaznu formulu je vazna njena interpretacija 
(istinitosna vrednost; tacno, netacno) u datoj valuaciji. 
Valuacija svakom iskaznom slovu pridrzuje neku istinitosnu vrednost (tacno, netacno).

Vise o ovome procitati u skripti (http://poincare.matf.bg.ac.rs/~janicic//courses/vi.pdf),
u delu "Sematika iskazne logike" (78 strana).

Glavni zadatak je da se za datu iskaznu formulu dobije valuacija u kojoj je formula tacna.
'''
class Formula():
    def __init__(self, args=None):
        self.args = args
        
# Logicka konstanta, tacno, te
class T(Formula):
    def __init__(self):
        super().__init__()
        
    # Funkcija za ispis formule
    def __str__(self):
        return u'\u22A4'
    
    # Interpretacija formule u datoj valuaciji je uvek tacno jer
    # je u pitanju logicka konstanta kojoj je istinitosna vrednost uvek tacno.
    def interpretation(self, valuation):
        return True

# Logicka konstanta, netacno, nete
class F(Formula):
    def __init__(self):
        super().__init__()
        
    def __str__(self):
        return u'\u22A5'
    
    def interpretation(self, valuation):
        return False

# Iskazne promenljive
class Letter(Formula):
    def __init__(self, character):
        super().__init__()
        self.character = character
        
    def __str__(self):
        return self.character
    
    # Interpretacija iskazne promenljive je jednaka valuaciji iskazne promenljive.
    def interpretation(self, valuation):
        return valuation[self.character]

# Negacija    
class Not(Formula):
    def __init__(self, operand):
        super().__init__([operand])
        
    # Koristi se unicode za ispis karaktera za negaciju        
    def __str__(self):
        return u'\u00AC({})'.format(self.args[0])
    
    # Interpetacija se odredjuje kao negacija interpretacije operanda
    def interpretation(self, valuation):
        return not self.args[0].interpretation(valuation)
    
# Konjukcija    
class And(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return u'(({}) \u2227 ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return self.args[0].interpretation(valuation) and self.args[1].interpretation(valuation)

# Disjunkcija
class Or(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return u'(({}) \u2228 ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return self.args[0].interpretation(valuation) or self.args[1].interpretation(valuation)
    
# Implikacija    
class Imp(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return u'(({}) \u21D2 ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return not self.args[0].interpretation(valuation) or self.args[1].interpretation(valuation)
    
# Ekvivalencija    
class Eq(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return u'(({}) \u21D4 ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return self.args[0].interpretation(valuation) == self.args[1].interpretation(valuation)

In [7]:
p = Letter('p')
q = Letter('q')
f1 = Not(p)
f2 = And(Or(T(), q), p)

valuation = {
    'p' : True,
    'q' : False
}

print(f1)
f1.interpretation(valuation)

print(f2)
f2.interpretation(valuation)

¬(p)
((((⊤) ∨ (q))) ∧ (p))


True

In [11]:
'''
Odredjivanje konjuktivne normalne forme za datu formulu.
Algoritam i njegove osobine se moze naci u skripti 
(http://poincare.matf.bg.ac.rs/~janicic//courses/vi.pdf), 
u delu "ormalne forme i potpuni skupovi veznika" (strana 86).
'''
def CNF(formula):
    if isinstance(formula, T) or isinstance(formula, F) or isinstance(formula, Letter):
        return formula
    
    if isinstance(formula, Eq):
        [A, B] = formula.args
        return CNF(And(
            Imp(A, B),
            Imp(B, A)
        ))
    
    if isinstance(formula, Imp):
        [A, B] = formula.args
        return CNF(Or(
            Not(A),
            B
        ))
    
    if isinstance(formula, Not):
        argF = CNF(formula.args[0])
        
        if isinstance(argF, And):
            [A, B] = argF.args
            return CNF(Or(
                Not(A),
                Not(B)
            ))
        
        if isinstance(argF, Or):
            [A, B] = argF.args
            return CNF(And(
                Not(A),
                Not(B)
            ))
        
        if isinstance(argF, Not):
            return CNF(argF.args[0])
        
        return Not(argF)
        
    if isinstance(formula, Or):
        [A, B] = formula.args
        
        A = CNF(A)
        B = CNF(B)
        
        if isinstance(A, And):
            return CNF(And(
                    Or(A.args[0], B),
                    Or(A.args[1], B)
                ))
            
        
        if isinstance(B, And):
            return CNF(And(
                    Or(A, B.args[0]),
                    Or(A, B.args[1])
                ))
            
        return Or(
            A,
            B
        )
    
    if isinstance(formula, And):
        return And(
            CNF(formula.args[0]),
            CNF(formula.args[1])
        )

In [14]:
f3 = Or(And(p, q), Or(Not(p), q))
print(f3)
print()
print("CNF:")
print(CNF(f3))

((((p) ∧ (q))) ∨ (((¬(p)) ∨ (q))))

CNF:
((((p) ∨ (((¬(p)) ∨ (q))))) ∧ (((q) ∨ (((¬(p)) ∨ (q))))))


In [15]:
r = Letter('r')

f = Imp(Eq(p,q),And(Not(p), r))

print(f)
print()
print("CNF:")
print(CNF(f))

((((p) ⇔ (q))) ⇒ (((¬(p)) ∧ (r))))

CNF:
((((((((((p) ∨ (q))) ∨ (¬(p)))) ∧ (((((p) ∨ (q))) ∨ (r))))) ∧ (((((((p) ∨ (¬(p)))) ∨ (¬(p)))) ∧ (((((p) ∨ (¬(p)))) ∨ (r))))))) ∧ (((((((((¬(q)) ∨ (q))) ∨ (¬(p)))) ∧ (((((¬(q)) ∨ (q))) ∨ (r))))) ∧ (((((((¬(q)) ∨ (¬(p)))) ∨ (¬(p)))) ∧ (((((¬(q)) ∨ (¬(p)))) ∨ (r))))))))


In [16]:
'''
Pomocne funkcije potrebne za odredjivanje DIMACS formata.
'''
# Niz literala koji se pojavljuju u klauzi
def get_literals(clause):
    if isinstance(clause, Or):
        return get_literals(clause.args[0]) + get_literals(clause.args[1])
    
    else:
        return [clause]

# Za datu formulu u CNF-u napravi niz klauza, a za svaku klazu, napravi niz literala
def get_clauses(cnf_formula):
    if isinstance(cnf_formula, And):
        return get_clauses(cnf_formula.args[0]) + get_clauses(cnf_formula.args[1])
    
    else:
        return [get_literals(cnf_formula)]

# U datoj formuli odredjuje sva iskazna slova koja se pojavljuju.
def all_letters(formula):
    if isinstance(formula, Letter):
        return [formula.character]
    
    else:
        if len(formula.args) == 1:
            return all_letters(formula.args[0])
        elif len(formula.args) == 2:
            # Da se iskazna slova ne bi ponavljala, lista se pretvori u skup, a potom vrati u listu
            return list(set(all_letters(formula.args[0]) + all_letters(formula.args[1])))
        else:
            return []    

In [26]:
klauza = Or(p, Or(Not(q), r))

print(klauza)
literali = get_literals(klauza)
letters = all_letters(klauza)

print()
print("Literali:")
for l in literali:
    print(l)
    
print()    
print('Iskazna slova:')    
for l in  letters:
    print(l)

((p) ∨ (((¬(q)) ∨ (r))))

Literali:
p
¬(q)
r

Iskazna slova:
p
r
q


In [33]:
cnf_formula = And(Or(p, q), klauza)

print(cnf_formula)

niz_klauza = get_clauses(cnf_formula)

print()
print("Ispis literala u klauzama:")
for k in niz_klauza:
    print('[', end="")
    for l in k:
        print('{}, '.format(l), end="")
    print(']')

((((p) ∨ (q))) ∧ (((p) ∨ (((¬(q)) ∨ (r))))))

Ispis literala u klauzama:
[p, q, ]
[p, ¬(q), r, ]


In [34]:
'''
Formula se zapisuje u DIMACS formatu koji je citljiviji i pogodniji za
koriscenje u okviru automatskih resavaca.

Opis dimacs formata pogledati u slajdovima sa casa (151 - 160 slajd) ili na
https://people.sc.fsu.edu/~jburkardt/data/cnf/cnf.html
'''
def dimacs(formula):
    formula_cnf = CNF(formula)
    
    letters = all_letters(formula_cnf)
    
    mapping = {}
    
    # Svakom iskaznom slovu se dodeljuje jedan broj
    for (index, letter) in enumerate(letters):
        mapping[letter] = index + 1
        
    clauses = get_clauses(formula_cnf)
    
    num_clauses = len(clauses)
    num_letters = len(letters)
    
    # Prva linija DIMACS formata
    header = 'p cnf {} {}\n'.format(num_letters, num_clauses)
    
    body = ''
    
    # Za svaku klauzu se pravi jedana linija (oblika 1 2 -3 0)
    for clause in clauses:
        line = ''
        
        for literal in clause:
            # Ukoliko je iskazno slovo ispisuje se njemu_dodeljen_broj
            if isinstance(literal, Letter):
                line += str(mapping[literal.character]) + ' '
            # Ako nije iskazno slovo, onda je Not(iskazno_slovo) i ispisuje se
            # minus njemu_dodeljen_broj
            else:
                line += str(- mapping[literal.args[0].character]) + ' '
        
        # Stavlja se 0 i novi red na kraj linije
        line += '0\n'
        body += line
        
    return header + body

In [38]:
# Zadato iznad
# f3 = Or(And(p, q), Or(Not(p), q))

print('DIMACS:')
print(dimacs(f3))

DIMACS:
p cnf 2 2
1 -1 2 0
2 -1 2 0



In [39]:
# Zadato iznad
# f = Imp(Eq(p,q),And(Not(p), r))

print('DIMACS:')
print(dimacs(f))



DIMACS:
p cnf 3 8
1 3 -1 0
1 3 2 0
1 -1 -1 0
1 -1 2 0
-3 3 -1 0
-3 3 2 0
-3 -1 -1 0
-3 -1 2 0



In [40]:
'''
Naivni resavac. Pronalazi valuaciju u kojoj je formula tacna.
'''
def SAT(formula, valuation, letters, i):
    if i == len(letters):
        if formula.interpretation(valuation):
            return valuation
        else:
            return False
        
    current_letter = letters[i]
    valuation[current_letter] = True
    
    res = SAT(formula, valuation, letters, i + 1)
    if res:
        return res
    
    current_letter = letters[i]
    valuation[current_letter] = False
    
    return SAT(formula, valuation, letters, i + 1)

In [41]:
letters = all_letters(f)
print('Trivial SAT solver:')
solution = SAT(f, {}, letters, 0)
print(solution)
print()

Trivial SAT solver:
{'p': True, 'r': True, 'q': False}

